In [ ]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from sklearn.model_selection import train_test_split
# 4. Model Training


In [ ]:
import pandas as pd
mobile = pd.read_csv('Data/mobile_cleaned.csv')
# mobile.drop(columns='time',inplace=True)
mobile.drop_duplicates(inplace=True)
mobile = mobile.reset_index(drop=True)
mobile.reset_index(drop=True, inplace=True)

print(mobile.mobile_strength.min())
print(mobile.mobile_strength.max())
# we have max and min
# we set a range interval based score for the strength
mobile.mobile_strength = mobile.mobile_strength.div(10)
mobile

In [ ]:
from sklearn.model_selection import train_test_split
X = mobile[['time', 'lat', 'long']]  # Features: time, latitude, longitude
y = mobile['mobile_strength']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.001, random_state=42)


In [ ]:
model = RandomForestRegressor()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)


df = pd.DataFrame({'lat':X_test.lat,'long':X_test.long,'expected':y_test,'predicted':y_pred}).reset_index(drop=True)
df

In [ ]:
print(X_test)

In [ ]:
from sklearn.neighbors import KDTree
higher_signal_points = mobile[mobile.mobile_strength>=df.predicted[0]+1].reset_index(drop=True)[['lat','long']]
print(higher_signal_points)
kd_tree = KDTree(higher_signal_points)

given_point = [[df.lat[0], df.long[0]]]

distance, nearest_index = kd_tree.query(given_point, k=1)

print(distance[0], nearest_index[0])

nearest_point = higher_signal_points.iloc[nearest_index[0][0]]
print(nearest_point)